# Send pickers to Kinesis

In this file we generate rand picker movements between blocks.

In [ ]:
import com.amazonaws.services.kinesis.model.PutRecordRequest
import com.amazonaws.services.kinesis.AmazonKinesisClientBuilder
import com.amazonaws.auth.{AWSStaticCredentialsProvider, BasicAWSCredentials}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import collection.JavaConverters._

import java.nio.ByteBuffer
import scala.util.Random

import scala.util.Random
import java.lang.reflect.{Type, ParameterizedType}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.annotation.JsonProperty;
import com.fasterxml.jackson.core.`type`.TypeReference;

In [ ]:
val awsAccessKeyId = "******"
val awsSecretKey = "*******"
val kinesisStreamName = "Jedha-pickers"
val kinesisRegion = "eu-west-1" 

sc.hadoopConfiguration.set("fs.s3a.access.key", awsAccessKeyId)
sc.hadoopConfiguration.set("fs.s3a.secret.key", awsSecretKey)

In [ ]:
case class Picker(block: Integer, action: String)


def getRandomPicker(action: String): Picker = {
  val r = scala.util.Random
  Picker(r.nextInt(10), action)
}

val kinesisClient = AmazonKinesisClientBuilder.standard()
  .withRegion(kinesisRegion)
  .withCredentials(new AWSStaticCredentialsProvider(new BasicAWSCredentials(awsAccessKeyId, awsSecretKey)))
  .build()

var lastSequenceNumber: String = null

val mapper = new ObjectMapper()
mapper.registerModule(DefaultScalaModule)

var overallNumberOfPickers = 0
val optimalNumberOfPickers = 70


for (i <- 0 to 100) {
  val time = System.currentTimeMillis
  var action = "add"
  
  if (overallNumberOfPickers < optimalNumberOfPickers){
    action = "add"
  } else {
    val r = scala.util.Random
    action = if (r.nextDouble()<0.5) "add" else "remove"
  }
  val data = mapper.writeValueAsString(getRandomPicker(action)) + "\n"
  
  val request = new PutRecordRequest()
      .withStreamName(kinesisStreamName)
      .withPartitionKey("some-key")
      .withData(ByteBuffer.wrap(data.getBytes()))
  if (lastSequenceNumber != null) {
    request.setSequenceNumberForOrdering(lastSequenceNumber)
  }    
  val result = kinesisClient.putRecord(request)
  println(result)
  lastSequenceNumber = result.getSequenceNumber()

  Thread.sleep(math.max(10000 - (System.currentTimeMillis - time), 0)) // loop around every ~10 seconds 
}